In [55]:
# IMPORT LIBRARIES #
import pandas as pd
import pygsheets
import numpy as np

In [2]:
# FLAGS #

#Set to 1 to print diagnostic info
debugFlag = 0
#Set to 1 to pull info from RawNames sheet, join GPS, update, and export
updateGPSFlag = 0

In [3]:
# IMPORT DATA #

gc = pygsheets.authorize(service_file='file.json')
sh = gc.open('DashData')

if debugFlag:
    for x in sh:
        print(x)

for x in sh:
    title = x.title
    if(title=="DeliveriesOld"):
        import_sheets_OldDelDF = x.get_as_df()
    elif(title=="Deliveries"):
        import_sheets_DelDF = x.get_as_df()
    elif(title=="Days"):
        import_sheets_DaysDF = x.get_as_df()
    elif(title=="Weeks"):
        import_sheets_WeeksDF = x.get_as_df()
    elif(title=="RawNames"):
        import_sheets_RawNamesDF = x.get_as_df()
    elif(title=="GPS"):
        #for import
        import_sheets_GPSDF = x.get_as_df()
        #for export
        import_wks_gps = x
    else:
        print("Unused worksheet detected: ", x)

Unused worksheet detected:  <Worksheet 'Import' index:6>
Unused worksheet detected:  <Worksheet 'Lifetime Deliveries' index:7>
Unused worksheet detected:  <Worksheet 'Days Old' index:8>
Unused worksheet detected:  <Worksheet 'Weeks Old' index:9>


In [34]:
# Allows for disjoint cells, one for import, one for reset to import value (for ease of testing)

#DF from sheets
sheets_OldDelDF   = import_sheets_OldDelDF.copy()
sheets_DelDF      = import_sheets_DelDF.copy()
sheets_DaysDF     = import_sheets_DaysDF.copy()
sheets_WeeksDF    = import_sheets_WeeksDF.copy()
sheets_RawNamesDF = import_sheets_RawNamesDF.copy()
sheets_GPSDF      = import_sheets_GPSDF.copy()

#Sheets, ah right can't use copy on a worksheet object, might need an alternate
#wks_gps           = import_wks_gps.copy()

In [35]:
# FUNCTIONS #

# TODO: develop function to take avg distance and duration for stacked orders

def dollars_to_floats(df, columns):
    for x in columns:
        df[x] = df[x].str.replace('$', '')
        df[x] = df[x].str.replace(',', '')
        df[x] = df[x].astype(float)
        
def hours_to_floats(df, columns):
    for x in columns:
        for index, row in df.iterrows():
            active = row[x]
            activeH = float(active[:active.index(":")])
            activeM = round(float(active[active.index(":")+1:active.index(":")+3])/60, 2)
            active = activeH + activeM
            df.at[index, x] = active
        df[x] = df[x].astype(float)
            
            

#Currently only valid for 2023
def date_to_timestamp(df, column):
    for index, row in df.iterrows():
        df.at[index, column] = pd.Timestamp('2023/' + row[column])
        
        
#Currently only valid for 2023
# Currently unused function
# Would we actually want this to have different dates? it's really the date the shift started
# So maybe, keep date same, but certain analysis might require that difference to be handled
def time_to_timestamp(df, DateCol, TimeCols):
    for index, row in df.iterrows():
        df.at[index, TimeCols[0]] = pd.Timestamp('2023/' + row[DateCol] + " " + row[TimeCols[0]])
        df.at[index, TimeCols[1]] = pd.Timestamp('2023/' + row[DateCol] + " " + row[TimeCols[1]])

In [36]:
# UPDATE GPS WORKSHEET #

if updateGPSFlag:
    testingDF = sheets_RawNamesDF.copy()
    testingDF = testingDF.drop_duplicates(subset=["Restaurant Name", "RAW DATA"])
    testingDF = testingDF[["Restaurant Name", "RAW DATA", "Note"]]
    testingDF = testingDF.sort_values(by=['Restaurant Name', "RAW DATA"])
    newNamesDF = testingDF.merge(sheets_GPSDF, how='left')
    for index, row in newNamesDF.iterrows():
        if isinstance(row['Address'], float):
            if not row['Note'] == "":
                newNamesDF.at[index, "Address"] = row["Note"]
            else:
                print("Note not found for:", index)
        if (row['Address'] == ""):
            newNamesDF.at[index, "Resolved"] = "0"
        else:
            newNamesDF.at[index, "Resolved"] = "1"
    newNamesDF = newNamesDF.drop(columns=["Note"])
    wks_gps.set_dataframe(newNamesDF,(1,1))

In [37]:
# COMBINE DELIVERY DATA #
sheets_frames = [sheets_OldDelDF, sheets_DelDF]
sheets_result = pd.concat(sheets_frames)
sheets_result.reset_index(drop=True, inplace=True)

In [38]:
# FUNCTIONS TO CONVERT DOLLAR AND HOUR VALUES TO FLOATS #


#All
dollars_to_floats(sheets_result, ["Total", "Base", "Tip", "Peak Bonus"])
#V3 only
dollars_to_floats(sheets_DelDF, ["Total", "Base", "Tip", "Peak Bonus"])
#Days
dollars_to_floats(sheets_DaysDF, ["Total", "Base", "Tip"])
hours_to_floats(sheets_DaysDF, ["Dash", "Active"])
#Weeks
dollars_to_floats(sheets_WeeksDF, ["Total", "Base", "Tip", "Adj pay", "Other", "Pre Total"])
hours_to_floats(sheets_WeeksDF, ["Dash", "Active"])

/tmp/ipykernel_24275/4229877212.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[x] = df[x].str.replace('$', '')


In [39]:
# BEGIN ADJ PAY SECTION #

# This establishes the estimated pay adjustment estimates by delivery, date, and week
# This is calculated using time spent delivering and distance approximations
# The real adjusted pay is based on weekly values
# I suspect the primary cause for difference between estimate and real values stems from 
#how I estimate mileage vs how the app tracks mileage

# DFs are copied to avoid excluding dropped values in later analysis



#if debugFlag:
#    sheets_DelDF.tail()

In [40]:
#Per delivery estimations
sheets_DelDF1 = sheets_DelDF.copy()

#Convert to float if possible, else convert to NaN
sheets_DelDF1["Duration"] = pd.to_numeric(sheets_DelDF1["Duration"], errors='coerce')
#Drop NaN rows
sheets_DelDF1 = sheets_DelDF1[sheets_DelDF1["Duration"].notna()]
#Calculate estimate adjustment per delivery
sheets_DelDF1["Est Adj"] = (sheets_DelDF1["Duration"]/60*18
                           + sheets_DelDF1["Distance"]*(0.34) ) - sheets_DelDF1["Base"]
#Duration is over 60 because its value is in minutes, values from day and week are in hours

#if debugFlag:
#    sheets_DelDF.tail()

In [41]:
#if debugFlag:
#    sheets_DaysDF.tail()

In [42]:
#Per day estimations
sheets_DaysDF1 = sheets_DaysDF.copy()
sheets_DaysDF1 = sheets_DaysDF1[sheets_DaysDF1["Version"] == 3]
sheets_DaysDF1["Est Adj"] = (sheets_DaysDF1["Active"]*(18) 
                            + sheets_DaysDF1["Active Mileage"]*(0.34) ) - sheets_DaysDF1["Base"]

In [43]:
#if debugFlag:
#    sheets_DaysDFv3.tail()

In [44]:
#Per week estimations
sheets_WeeksDF1 = sheets_WeeksDF.copy()
sheets_WeeksDF1 = sheets_WeeksDF1[sheets_WeeksDF1["Version"] == 3] #totals doesn't have v3 so its dropped
sheets_WeeksDF1["Est Adj"] = (sheets_WeeksDF1["Active"]*(18) 
                            + sheets_WeeksDF1["Active Mileage"]*(0.34) ) - sheets_WeeksDF1["Base"]

In [45]:
#if debugFlag:
#    sheets_WeeksDFv3.tail()

# Sample values

print("Del DF Sample\n")
print(sheets_DelDF1.iloc[-1])
print("\nDays DF Sample\n")
print(sheets_DaysDF1.iloc[-1])
print("\nWeeks DF Sample\n")
print(sheets_WeeksDF1.iloc[-1])
    
    
# END ADJ PAY SECTION #

Del DF Sample

ID                       1026
Date                     7/29
Restaurant Name    McDonald's
Total                     9.0
Base                      5.5
Tip                       3.5
Peak Bonus                0.0
Stacked                      
Start Time         6:41:00 PM
Distance                  2.1
End Time           7:01:00 PM
Duration                 20.0
Version                     3
Est Adj                 1.214
Name: 451, dtype: object

Days DF Sample

Date                7/29
Total             111.25
Base               66.75
Tip                 44.5
Start (24)         18:00
End (24)           20:10
Active              3.55
Dash                4.15
Offers                10
Deliveries            12
Mileage Start      71086
Mileage End        71149
Total Mileage         63
Active Mileage      84.3
Version                3
Pauses                 0
Est Adj           25.812
Name: 57, dtype: object

Weeks DF Sample

Dates             7/24 - 7/30
Base                   278

In [46]:
#Percentage of deliveries taken that tip
numUntipped = (sheets_result["Tip"] == 0.0).sum()
numTipped = (sheets_result["Tip"] != 0.0).sum()
percTipped = round((numTipped/(numUntipped+numTipped)), 4)
print(percTipped*100, "% of customers tip")

90.63 % of customers tip


In [75]:
# Restaurants rankable by avg or mean base/tip/total
# Yes, total is almost always base+tip, but total INCLUDES peak pay, which may be desirable information


tempDF = sheets_result.copy()
tempDF = tempDF[sheets_result["Restaurant Name"].duplicated(keep=False) == True]
tempDF = tempDF.groupby(["Restaurant Name"]).agg({"Base": [np.mean, np.sum], 
                                                  "Tip": [np.mean, np.sum],
                                                  "Total": [np.mean, np.sum]})
tempDF.columns = ['Avg Base', 'Sum Base', 'Avg Tip', 'Sum Tip', 'Avg Total', 'Sum Total']

tempDF = tempDF.round({'Avg Base': 2, 
                       'Sum Base': 2,
                       'Avg Tip': 2,
                       'Sum Tip': 2,
                       'Avg Total': 2,
                       'Sum Total': 2})

#Change sort value to see differences
tempDF.sort_values("Sum Total", ascending=False)

,Avg Base,Sum Base,Avg Tip,Sum Tip,Avg Total,Sum Total
Restaurant Name,,,,,,
Chick-fil-A,5.70,672.50,3.67,433.39,9.37,1105.89
Taco Bell,4.79,306.62,3.46,221.75,8.75,559.87
McDonald's,4.59,316.50,3.30,227.42,7.90,544.92
Dave's Hot Chicken,5.20,244.50,4.46,209.50,9.66,454.00
Chipotle,4.65,223.25,3.62,173.92,8.32,399.17
...,...,...,...,...,...,...
Wingstop Drive,5.00,10.00,2.90,5.79,7.90,15.79
Jersey Mike's,3.75,7.50,3.50,7.00,7.75,15.50
Rancho Viejo Mexican Food 2,4.38,8.75,3.25,6.50,7.62,15.25


In [48]:
# Overall hourly rate

hourlyRate = round(sheets_WeeksDF.iloc[-1]["Total"]/sheets_WeeksDF.iloc[-1]["Dash"], 2)
print("$", hourlyRate, "per hour, overall average")

$ 25.47 per hour, overall average


In [49]:
# Active time is time spent on deliveries, dash time is time spent on deliveries AND waiting for deliveries
# One factor that I wanted to consider in possible earnings maximization is:
# (Base + Tip) / Active time
# I've referred to this as the Pre Total : Active ratio
# Instead of something sensible like, active hourly pay rate

sheets_WeeksDF["pretotalactiveratio"] = round(sheets_WeeksDF["Pre Total"]/sheets_WeeksDF["Active"], 2)
sheets_WeeksDF.sort_values(by=["pretotalactiveratio"])

,Dates,Base,Tip,Adj pay,Other,Pre Total,Total,Active,Dash,Deliveries,Total Mileage,Active Mileage,Version,pretotalactiveratio
2,4/10 - 4/16,130.75,96.13,61.07,0.00,226.88,287.95,8.63,11.17,28,,,1,26.29
13,7/3 - 7/9,114.25,105.10,54.99,7.75,219.35,282.09,7.83,10.17,23,108,98.1,3,28.01
4,4/24 - 4/30,430.00,299.73,138.76,2.62,729.73,871.11,25.50,41.17,84,,,1,28.62
12,6/19 - 6/25,637.25,475.09,240.02,6.25,1112.34,1358.61,38.60,51.23,124,633,619.4,3,28.82
0,3/27 - 4/2,108.50,88.00,40.96,0.00,196.50,237.46,6.70,10.92,22,,,1,29.33
3,4/17 - 4/23,382.50,248.41,105.05,0.00,630.91,735.96,21.33,32.30,68,,,1,29.58
7,5/15 - 5/21,303.75,239.25,99.27,0.00,543.00,642.27,17.53,25.40,64,,,2,30.98
16,TOTALS:,5342.50,4215.67,1577.76,32.75,9558.17,11168.68,304.08,438.52,1024,2446,2229.68,-,31.43
10,6/5 - 6/11,197.25,185.09,65.38,0.00,382.34,447.72,11.77,15.75,41,183,161.6,3,32.48
6,5/8 - 5/14,758.25,567.84,155.48,6.25,1326.09,1487.82,40.58,58.13,138,,,2,32.68


In [50]:
#Dollars Earned vs Miles Driven is another factor I wanted to consider in the overall earnings equation

# Grab v3 deliveries, drop rows without known distance values
# Group by restaurant name, sum on total and distance
# Evaluate total pay over total distance as new column
# Sort low to high



tempDF = sheets_DelDF.copy()
tempDF = tempDF[tempDF.Distance!="-"]
tempDF["Distance"] = tempDF["Distance"].astype(float)
tempDF = tempDF.groupby(["Restaurant Name"]).agg({"Total": "sum", "Distance": "sum"})
tempDF = tempDF.round({'Base': 2, 'Tip': 2})
tempDF["DollarMilesRatio"] = tempDF["Total"]/tempDF["Distance"]
tempDF.sort_values(by="DollarMilesRatio")

,Total,Distance,DollarMilesRatio
Restaurant Name,,,
Yukiya Sushi,8.25,10.8,0.763889
Pedro's Tacos,7.00,7.9,0.886076
El Bronco,9.25,9.7,0.953608
Papa Johns,5.00,4.9,1.020408
Urban Plates,21.50,20.4,1.053922
...,...,...,...
The Privateer Coal Fire Pizza,8.00,1.8,4.444444
Ohana Hawaiian BBQ,8.25,1.8,4.583333
Round Table Pizza,9.75,1.4,6.964286


In [67]:
#Need to drop non numerics to get max 'Total Mileage' - right now its 151 miles
sheets_DaysDF[["Total", "Base", "Tip", "Active", "Dash", "Deliveries"]].max() 
#This isn't useful for later insights, but I find it interesting

/tmp/ipykernel_24275/54442025.py:1: FutureWarning: The default value of numeric_only in DataFrame.max is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  sheets_DaysDF[["Total", "Base", "Tip", "Active", "Dash", "Deliveries", "Total Mileage"]].max()


Total         333.57
Base          191.50
Tip           149.53
Active          9.67
Dash           13.60
Deliveries     33.00
dtype: float64

In [52]:
# Also interested in longest order by duration and by distance
# Using sheets_DelDF1 because it has the nan value dropped
sheets_DelDF1[["Distance", "Duration"]].max()

Distance    17.0
Duration    67.0
dtype: float64

In [53]:
#Average tip, including and excluding untipped deliveries

TipAvg = round(sheets_result["Tip"].mean(), 2)
print("$", TipAvg, "average tip value including orders with no tip")
TipAvg2 = round(sheets_result[sheets_result["Tip"] !=0]["Tip"].mean(), 2)
print("$", TipAvg2, "average tip value excluding orders with no tip")

#This isn't useful for later insights, but I find it interesting

$ 4.11 average tip value including orders with no tip
$ 4.54 average tip value excluding orders with no tip


In [66]:
# Average $/hour and tip amount by day of week
# NOTE: This excludes pay adjustments

tempDF = sheets_DaysDF.copy()

# Can move this up to formatting function calls
# Also, check if necessary to convert from string to timestamp to datetime
date_to_timestamp(tempDF, "Date")
tempDF["Date"] = pd.to_datetime(tempDF["Date"])

#Adds column to DaysDF for the day of the week (Monday, Thursday, etc.)
tempDF["Day of Week"] = tempDF["Date"].dt.day_name()

tempDF = tempDF.groupby(["Day of Week"]).agg(
    {"Dash": "sum", 
     "Total": [np.mean, np.sum], 
     "Tip": "sum", 
     "Deliveries": "sum"})
tempDF.columns = ['Sum Dash', 'Avg Total', 'Sum Total', 'Sum Tip', 'Sum Deliveries']

tempDF["$/hr"] = tempDF["Sum Total"]/tempDF["Sum Dash"]
tempDF["AvgTip"] = tempDF["Sum Tip"]/tempDF["Sum Deliveries"]
tempDF["AvgTotal"] = tempDF["Sum Total"]/tempDF["Sum Deliveries"]

tempDF = tempDF.round({'Sum Dash': 2, 
                       'Avg Total': 2, 
                       'Sum Total':2, 
                       'Sum Tip': 2,
                       '$/hr': 2,
                       'AvgTip': 2,
                       'AvgTotal':2})

display(tempDF)

,Sum Dash,Avg Total,Sum Total,Sum Tip,Sum Deliveries,$/hr,AvgTip,AvgTotal
Day of Week,,,,,,,,
Friday,127.99,191.27,2869.09,1318.34,301,22.42,4.38,9.53
Monday,17.85,125.82,377.47,158.97,41,21.15,3.88,9.21
Saturday,106.08,186.36,2422.70,1067.45,252,22.84,4.24,9.61
Sunday,45.36,154.83,1083.78,507.03,111,23.89,4.57,9.76
Thursday,72.83,140.77,1548.51,656.01,176,21.26,3.73,8.80
Wednesday,67.81,139.62,1256.62,507.87,143,18.53,3.55,8.79
